In [1]:
import numpy as np
import pandas as pd

pheno_train = pd.read_csv("train/pheno_training.csv")
readcounts_train = pd.read_csv("train/readcounts_training.csv")
taxtable_train = pd.read_csv("train/taxtable.csv")

pheno_test = pd.read_csv("test/pheno_test.csv")
readcounts_test = pd.read_csv("test/readcounts_test.csv")
taxtable_test = pd.read_csv("test/taxtable.csv")

pheno_train = pheno_train.dropna()
pheno_test = pheno_test.dropna()

## Pré processamento para ML

In [2]:
pheno_train_y = pheno_train.PrevalentHFAIL
pheno_train_X = pheno_train.iloc[: , 1:]
pheno_train_X = pheno_train_X.loc[:, pheno_train_X.columns != "PrevalentHFAIL"]
pheno_test_y = pheno_test.PrevalentHFAIL
pheno_test_X = pheno_test.iloc[: , 1:]
pheno_test_X = pheno_test_X.loc[:, pheno_test_X.columns != "PrevalentHFAIL"]


## Modelos de ML
#### Logistic Regression

In [3]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score

logistic = linear_model.LogisticRegression(C=1e5, solver = "liblinear", multi_class = "auto")
logistic = logistic.fit(pheno_train_X, pheno_train_y)
logistic_prev = logistic.predict(pheno_test_X)

print("PECC (accuracy):", accuracy_score(pheno_test_y, logistic_prev))

PECC (accuracy): 1.0


#### Linear Regression

In [4]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

regr_model = linear_model.LinearRegression()
regr_model = regr_model.fit(pheno_train_X, pheno_train_y)
regr_model_pred = regr_model.predict(pheno_test_X)

print(regr_model_pred)

#print("PECC (accuracy):", accuracy_score(pheno_test_y, regr_model_pred))
print("Não funciona com boleans")

[-0.05287802  0.08292254 -0.03657902 ...  0.24002286  0.00915692
 -0.03806369]
Não funciona com boleans


#### Ridge


In [5]:
ridge = linear_model.Ridge(alpha=.1)
ridge = ridge.fit(pheno_train_X, pheno_train_y)
ridge_pred = ridge.predict(pheno_test_X)

print(ridge_pred)
#print("PECC (accuracy):", accuracy_score(pheno_test_y, ridge_pred))
print("Não funciona com boleans")

[-0.0528788   0.0829221  -0.03657104 ...  0.24002048  0.00915614
 -0.03806499]
Não funciona com boleans


Lasso

In [13]:
lasso = linear_model.Lasso()
lasso = lasso.fit(pheno_train_X, pheno_train_y)
lasso_pred = lasso.predict(pheno_test_X)
print(lasso_pred)
#print("PECC (accuracy):", accuracy_score(pheno_test_y, lasso_pred))
print("Não funciona com boleans")

[0.0259887 0.0259887 0.0259887 ... 0.0259887 0.0259887 0.0259887]
Perc. acertos:  0.0
Não funciona com boleans


#### SVM's

In [15]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
SVM = svm.SVC(gamma=0.001, C=100.)
SVM = SVM.fit(pheno_train_X, pheno_train_y)
svm_pred = SVM.predict(pheno_test_X)
print("PECC (accuracy):", accuracy_score(pheno_test_y, svm_pred))
#cross_scores = cross_val_score(SVM, pheno_test_y, svm_pred, cv = 5)
#print(cross_scores)


PECC (accuracy): 1.0


#### KNeighborsClassifeir

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(pheno_train_X, pheno_train_y)
knn_pred = knn.predict(pheno_test_X)
print("Perc. acertos: ", (knn_pred == pheno_test_y).sum()/len(knn_pred))

Perc. acertos:  0.9971655328798186


#### Decision Tree

In [14]:
from sklearn import tree
from sklearn.metrics import f1_score

tree_model = tree.DecisionTreeClassifier()
tree_model = tree_model.fit(pheno_train_X, pheno_train_y)
preds_tree = tree_model.predict(pheno_test_X)

print("Perc. acertos: ", (preds_tree == pheno_test_y).sum()/len(preds_tree))


Perc. acertos:  1.0


## Ensembles
#### Voting Classifier

In [17]:
from sklearn.ensemble import VotingClassifier
DTree = tree.DecisionTreeClassifier(max_depth=4)
knn_class = KNeighborsClassifier(n_neighbors=7)
svc = svm.SVC(gamma='scale', kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('dt', DTree), ('knn', knn_class), ('svc', svc)], voting='hard')

eclf = eclf.fit(pheno_train_X, pheno_train_y)
print(eclf.score(pheno_test_X, pheno_test_y))

0.9965986394557823
